# Detección de anomalias usando Autoencoders

Referencia [link](https://keras.io/examples/timeseries/timeseries_anomaly_detection/)

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv1DTranspose, Dropout, LSTM, RepeatVector, TimeDistributed
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Datos

Usaremos el conjunto de datos [Numenta Anomaly Benchmark (NAB)](https://www.kaggle.com/boltzmannbrain/nab). Este proporciona series de tiempo artificiales que contienen períodos anómalos de comportamiento etiquetados. Los datos están ordenados, tienen marcas de tiempo y son métricas de un solo valor.

Usaremos el archivo `art_daily_small_noise.csv` para el entrenamiento y el archivo `art_daily_jumpsup.csv` para las pruebas. La simplicidad de este conjunto de datos nos permite demostrar la detección de anomalías de manera efectiva.



In [ ]:
master_url_root = "https://raw.githubusercontent.com/numenta/NAB/master/data/"

df_small_noise_url_suffix = "artificialNoAnomaly/art_daily_small_noise.csv"
df_small_noise_url = master_url_root + df_small_noise_url_suffix
df_small_noise = pd.read_csv(
    df_small_noise_url, parse_dates=True, index_col="timestamp"
)

df_daily_jumpsup_url_suffix = "artificialWithAnomaly/art_daily_jumpsup.csv"
df_daily_jumpsup_url = master_url_root + df_daily_jumpsup_url_suffix
df_daily_jumpsup = pd.read_csv(
    df_daily_jumpsup_url, parse_dates=True, index_col="timestamp"
)

In [ ]:
df_small_noise.head()

In [ ]:
df_daily_jumpsup.head()

## Visualización de los datos

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 6))
df_small_noise.plot(legend=False, ax=ax[0], title="Series sin anomalías")
df_daily_jumpsup.plot(legend=False, ax=ax[1], title="Series con anomalías")
plt.tight_layout()
plt.show()

## Preprocesado

Obtenemos los valores del archivo de datos de la serie de tiempo de entrenamiento y normalizamos los datos de `value`. Tenemos un `value` cada 5 minutos durante 14 días.

* 24 * 60 / 5 = **288 pasos de tiempo por día**
* 288 * 14 = **4032 puntos de datos** en total"


In [ ]:
training_mean = df_small_noise.mean()
training_std = df_small_noise.std()
df_training_value = (df_small_noise - training_mean) / training_std
print("Número de ejemplos de entrenamiento:", len(df_training_value))

## Creación de dataset

In [ ]:
TIME_STEPS = 288

def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

x_train = create_sequences(df_training_value.values)
print("Shape entrenamiento: ", x_train.shape)

## Construcción de modelos

In [ ]:
lstm_autoencoder = Sequential([
    Input(shape=(x_train.shape[1], x_train.shape[2])),
    LSTM(64, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(16, activation="relu", return_sequences=False),
    RepeatVector(x_train.shape[1]),
    LSTM(32, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(64, activation="relu", return_sequences=True),
    TimeDistributed(Dense(x_train.shape[2]))
])

lstm_autoencoder.compile(
    optimizer=Adam(learning_rate=5e-3),
    loss="mse",
)

lstm_autoencoder.summary()

In [ ]:
conv_autoencoder = Sequential([
    Input(shape=(x_train.shape[1], x_train.shape[2])),
    Conv1D(
        filters=32,
        kernel_size=7,
        padding="same",
        strides=2,
        activation="relu",
    ),
    Dropout(rate=0.2),
    Conv1D(
        filters=16,
        kernel_size=7,
        padding="same",
        strides=2,
        activation="relu",
    ),
    Dropout(rate=0.2),
    Conv1D(
        filters=16,
        kernel_size=7,
        padding="same",
        strides=2,
        activation="relu",
    ),
    Dropout(rate=0.2),
    Conv1DTranspose(
        filters=32,
        kernel_size=7,
        padding="same",
        strides=2,
        activation="relu",
    ),
    Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
])

conv_autoencoder.compile(
    optimizer=Adam(learning_rate=5e-3),
    loss="mse",
)

conv_autoencoder.summary()

## Entrenamiento

In [ ]:
history_1 = lstm_autoencoder.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

In [ ]:
history_2 = conv_autoencoder.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

In [ ]:
plt.plot(history_1.history["loss"], label="LSTM - Pérdida de Entrenamiento")
plt.plot(history_1.history["val_loss"], label="LSTM - Pérdida de Validación")
plt.legend()
plt.show()

In [ ]:
plt.plot(history_2.history["loss"], label="Conv - Pérdida de Entrenamiento")
plt.plot(history_2.history["val_loss"], label="Conv - Pérdida de Validación")
plt.legend()
plt.show()

## Detectando anomalías

Detectaremos anomalías determinando qué tan bien nuestro modelo puede reconstruir los datos de entrada.

1. Calcular la pérdida MAE en las muestras de entrenamiento.
2. Encontrar el valor máximo de la pérdida MAE. Este es el peor desempeño de nuestro modelo al intentar reconstruir una muestra. Usaremos este valor como el `umbral` para la detección de anomalías.
3. Si la pérdida de reconstrucción para una muestra es mayor que este valor `umbral`, entonces podemos inferir que el modelo está observando un patrón con el que no está familiarizado. Etiquetaremos esta muestra como una `anomalía`.


In [ ]:
# Obtenemos el MAE.
def calculate_mae(model, x_data, mode="entrenamiento"):
    x_data_pred = model.predict(x_data)
    mae_loss = np.mean(np.abs(x_data_pred - x_data), axis=1)
    plt.hist(mae_loss, bins=50)
    plt.xlabel("MAE de {}".format(mode)))
    plt.ylabel("Número de muestras")
    plt.show()
    threshold = np.max(mae_loss)
    return x_data_pred, threshold, mae_loss

In [ ]:
lstm_train_pred, lstm_threshold, _ = calculate_mae(lstm_autoencoder, x_train)
conv_train_pred, conv_threshold, _ = calculate_mae(conv_autoencoder, x_train)

### Comparar reconstrucción

Veamos cómo nuestro modelo ha reconstruido la primera muestra.
Esta corresponde a los **288 pasos de tiempo del día 1** de nuestro conjunto de entrenamiento.



In [ ]:
plt.plot(x_train[0])
plt.plot(lstm_train_pred[0])
plt.title("Reconstrucción con autoencoder LSTM")
plt.show()

In [ ]:
plt.plot(x_train[0])
plt.plot(conv_train_pred[0])
plt.title("Reconstrucción con autoencoder Conv")
plt.show()

### Preparación de los datos test

In [ ]:
df_test_value = (df_daily_jumpsup - training_mean) / training_std
x_test = create_sequences(df_test_value.values)
print("Shape testeo: ", x_test.shape)

## Métricas de autoencoders

In [ ]:
lstm_test_pred, _, lstm_test_mae = calculate_mae(lstm_autoencoder, x_test, mode="testeo")
lstm_test_mae = lstm_test_mae.reshape((-1))
anomalies = lstm_test_mae > lstm_threshold
plt.hist(lstm_test_mae, bins=50)
plt.xlabel("MAE de testeo")
plt.ylabel("Número de muestras")
plt.show()
print("Número de muestras anómalas: ", np.sum(anomalies))
print("Índices de las muestras anómalas: ", np.where(anomalies))

In [ ]:
conv_test_pred, _, conv_test_mae = calculate_mae(conv_autoencoder, x_test, mode="testeo")
conv_test_mae = conv_test_mae.reshape((-1))
anomalies = conv_test_mae > conv_threshold
plt.hist(conv_test_mae, bins=50)
plt.xlabel("MAE de testeo")
plt.ylabel("Número de muestras")
plt.show()
print("Número de muestras anómalas: ", np.sum(anomalies))
print("Índices de las muestras anómalas: ", np.where(anomalies))